In [65]:
DATA_PATH=r'..\data\processed\new_data.pickle'
MLFLOW_TRACKING_URI = '../models/mlruns/'
MLFLOW_EXPERIMENT_NAME = "Amazon_products_recommendation_system"

LOG_PATH = "../models/temp/"
LOG_DATA_PKL    =  "data.pkl"
LOG_MODEL_PKL   =  "model.pkl"
LOG_METRICS_PKL =  "metrics.pkl"

In [66]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import mlflow
from mlflow import MlflowClient
from pathlib import Path

In [67]:
df=pd.read_pickle(DATA_PATH)
df.head()

,User_ID,Product_ID,Ratings,year,month
1309,A3LDPF5FMB782Z,1400501466,5.0,2012,5
1321,A1A5KUIIIHFF4U,1400501466,1.0,2012,3
1334,A2XIOXRRYX0KZY,1400501466,3.0,2013,6
1450,AW3LX47IHPFRL,1400501466,5.0,2012,6
1455,A1E3OB6QMBKRYZ,1400501466,1.0,2012,10


In [68]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
client=MlflowClient()
mlflow.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)
exp=client.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)
if exp is None:
    print("Experiment not found ,creating new experiment")
    experiment_id=mlflow.create_experiment(MLFLOW_EXPERIMENT_NAME)
else:
    experiment_id=exp.experiment_id  

Rank-based recommendation sysetm 

In [69]:
average_rating = df.groupby('Product_ID')['Ratings'].mean()

# Calculate the count of ratings for each product
count_rating = df.groupby('Product_ID')['Ratings'].count()

# Create a dataframe with calculated average and count of ratings
final_rating = pd.DataFrame({'avg_rating':average_rating, 'rating_count':count_rating})

# Sort the dataframe by average of ratings in the descending order
final_rating = final_rating.sort_values(by='avg_rating', ascending=False)

# See the first five records of the "final_rating" dataset
final_rating.head(5)

,avg_rating,rating_count
Product_ID,,
B00LGQ6HL8,5.0,5
B004Y1AYAC,5.0,10
B004Z0S6RU,5.0,8
B000BOLHH0,5.0,5
B0051QZEHC,5.0,5


In [70]:
Path(MLFLOW_TRACKING_URI).mkdir(parents=True, exist_ok=True)
Path(LOG_PATH).mkdir(parents=True, exist_ok=True) 

In [71]:
final_rating.shape

(5523, 2)

In [72]:
# Defining a function to get the top n products based on the highest average rating and minimum interactions
def top_n_products(data, n, min_interaction=100):

    # Finding products with minimum number of interactions
    recommendations = data[data['rating_count'] > min_interaction]

    return recommendations.index[:n]
print(list(top_n_products(final_rating,5)))

['B003ES5ZUU', 'B000N99BBC', 'B002V88HFE', 'B007WTAJTO', 'B004CLYEDC']


In [73]:
top_products=list(top_n_products(final_rating,5))
for i in top_products:
    print(final_rating.loc[i])

avg_rating        4.858757
rating_count    177.000000
Name: B003ES5ZUU, dtype: float64
avg_rating        4.773006
rating_count    163.000000
Name: B000N99BBC, dtype: float64
avg_rating        4.70297
rating_count    101.00000
Name: B002V88HFE, dtype: float64
avg_rating        4.692308
rating_count    156.000000
Name: B007WTAJTO, dtype: float64
avg_rating        4.666667
rating_count    117.000000
Name: B004CLYEDC, dtype: float64


Collaporative Filtiring Recommendation System

Building a baseline user-user similarity based recommendation system¶



In [162]:
from surprise import Dataset,Reader
from surprise.model_selection import train_test_split
from surprise.prediction_algorithms.knns import KNNBaseline
from surprise import accuracy as acc
from surprise.model_selection import GridSearchCV
import os

In [152]:
reader=Reader(rating_scale=(1,5))
data=Dataset.load_from_df(df[['User_ID','Product_ID','Ratings']], reader)
trainset,testset=train_test_split(data,test_size=.2, random_state=42)

In [76]:
data

Building the user-user Similarity-based Recommendation System

In [113]:
sim_options={'name':'cosine',
             'user_based':True}
base_model=KNNBaseline(sim_options=sim_options)
base_model.fit(trainset=trainset)
predictions=base_model.test(testset=testset)
accuracy=acc.rmse(predictions)
print(accuracy)

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9415
0.9415216295051958


In [78]:
predictions

[Prediction(uid='AKBVYIIHWI04B', iid='B009SYZ8OC', r_ui=5.0, est=4.761802398863393, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid='A2MCRCK1V61FWQ', iid='B0009ZBRS0', r_ui=5.0, est=2.2897962321539085, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid='A27B1U3OWCU14J', iid='B005DKZW2I', r_ui=4.0, est=4.31629248980097, details={'actual_k': 0, 'was_impossible': False}),
 Prediction(uid='A2PZXXPGLXXKZU', iid='B0036E8V08', r_ui=4.0, est=3.974312389581907, details={'actual_k': 0, 'was_impossible': False}),
 Prediction(uid='A1ZBEIL78MLR9Z', iid='B00DR6DQZA', r_ui=5.0, est=4.561626430985831, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='A6FIAB28IS79', iid='B001FA1NZK', r_ui=4.0, est=5, details={'actual_k': 4, 'was_impossible': False}),
 Prediction(uid='A1P0JSFW184ZXJ', iid='B00I3ERXWS', r_ui=5.0, est=4.100167699260344, details={'actual_k': 8, 'was_impossible': False}),
 Prediction(uid='A2LEIANN1UZTHP', iid='B0044DEDC0', r_ui=2.0, est

we want to calculate percicion@k ,recall@k and f1@k

In [155]:
from collections import defaultdict
def metrices(model,k=10,threshold=3.5):
    user_est_true=defaultdict(list)
    predictions = model.test(testset)

    for uid,_,r_ui,est,_ in predictions:
        # print(uid,r_ui,est)
        user_est_true[uid].append((est,r_ui))

        
    precisions=dict()
    recalls=dict()
    # user_est_true
    for uid,user_ratings in user_est_true.items():
        # print(uid,user_ratings)
        user_ratings.sort(key=lambda x :x[0], reverse=True)
        # print(uid,user_ratings)
        # print('\n')
        # threshold=3.5
        #number of relevant items
        num_of_relevant=sum((r_ui>=threshold) for (_,r_ui)in user_ratings)
        
        #number of recommended items in top k
        # k=10
        num_of_recommended=(sum((est>=threshold) for (est,_) in user_ratings[:k]))
        
        #number of relevant and recommended

        num_of_both=(sum(  (r_ui>=threshold) and (est>=threshold) for (r_ui,est) in user_ratings[:k]   ))

        precisions[uid]=num_of_both/num_of_recommended if num_of_recommended!=0 else 0

        recalls[uid]=num_of_both/num_of_relevant if num_of_relevant!=0 else 0

    precision=round((sum(prec for prec in precisions.values()) / len(precisions)), 3)
    recall = round((sum(rec for rec in recalls.values()) / len(recalls)), 3)
    f1_score=round(2*precision*recall/(precision+recall),3)
    # accuracy=accuracy.rmse(predictions)
    return precision,recall,f1_score

In [96]:
print(metrices(base_model))

(0.854, 0.853, 0.853)


In [97]:
precision,recall,f1=metrices(base_model)

In [35]:
len(predictions)

12579

In [106]:
test_data = pd.DataFrame(testset, columns=['User_ID', 'Product_ID', 'Ratings'])
test_data

,User_ID,Product_ID,Ratings
0,AKBVYIIHWI04B,B009SYZ8OC,5.0
1,A2MCRCK1V61FWQ,B0009ZBRS0,5.0
2,A27B1U3OWCU14J,B005DKZW2I,4.0
3,A2PZXXPGLXXKZU,B0036E8V08,4.0
4,A1ZBEIL78MLR9Z,B00DR6DQZA,5.0
...,...,...,...
12574,AIJQU979J6UFY,B00JP12170,4.0
12575,ATOKT8QYK967L,B005O7LJAE,2.0
12576,AK3GKIV8DEY8B,B000067O7T,3.0
12577,ARXU3FESTWMJJ,B001TH7GVE,5.0


In [108]:
(base_model)

In [110]:
model = {"model_description": "Knnbaseline: cosine similarity ----user-based",
         "model_details": str(base_model),
         "model_object": base_model}

with open(os.path.join(LOG_PATH, LOG_MODEL_PKL), "wb") as output_file:
    pickle.dump(model, output_file)

In [114]:
metrices1={'accuracy: ':accuracy,
    'precision: ':precision,
    'recall: ' :recall,
    'f1_score: ':f1
          }
with open (os.path.join(LOG_PATH,LOG_METRICS_PKL),'wb') as output_file:
    pickle.dump(metrices1,output_file)

In [115]:
data1={'train_data : ':trainset,
      'test_data : ':testset  }
with open (os.path.join(LOG_PATH,LOG_DATA_PKL),'wb') as file:
    pickle.dump(data1,file)

In [117]:
with mlflow.start_run(experiment_id=exp.experiment_id,
                      run_name=model["model_description"]):
    # Log pickles
    mlflow.log_artifacts(LOG_PATH)

    # Track metrics
    mlflow.log_metric('precision', precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("accuracy", accuracy)

In [140]:
# Predicting rating for a sample user with an interacted product
sample=test_data.sample(1).iloc[0]
base_model.predict(sample["User_ID"], sample["Product_ID"], r_ui=sample["Ratings"], verbose=True)

user: A11I1I9QLMAM1A item: B00E3FHXYO r_ui = 5.00   est = 5.00   {'actual_k': 9, 'was_impossible': False}


Prediction(uid='A11I1I9QLMAM1A', iid='B00E3FHXYO', r_ui=5.0, est=5, details={'actual_k': 9, 'was_impossible': False})

Hyperparameter Tuning

In [153]:

param_grid={'k':[10,20,30,40],
            'min_k':[3,6,9],
            'sim_options':{
                'name' :['msd','cosine','pearson'],
                           'user_based':[True]
                           }
            }
grid=GridSearchCV(KNNBaseline,param_grid=param_grid,measures=['rmse'],cv=5,n_jobs=-1)
grid.fit(data)
print(grid.best_score)
print(grid.best_params)

{'rmse': 0.896583263001412}
{'rmse': {'k': 30, 'min_k': 9, 'sim_options': {'name': 'pearson', 'user_based': True}}}


In [154]:
best_params=grid.best_params

Taraining model on best Parms

In [170]:
# from surprise import accuracy
sim_options ={'name': 'pearson', 'user_based': True}
model1=KNNBaseline(k=30,min_k=9,sim_options=sim_options)
model1.fit(trainset)
predictions=model1.test(testset)
rmse1=acc.rmse(predictions)
precision,recall,f1=metrices(model1)

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.8887


In [167]:
acc.rmse(predictions)

RMSE: 0.8887


0.8886810203488327

In [165]:
print(precision)
print(recall)
print(f1)

0.847
0.893
0.869


In [161]:
model1

In [176]:
model2={'model_description':"KNNBaseline with grid search----user-based ",
        'model_details': str(model1),
        'model_object':model1}
with open (os.path.join(LOG_PATH,LOG_MODEL_PKL),'wb') as output_file:
    pickle.dump(model2,output_file)

In [177]:
metrices2={'accuracy: ':rmse1,
    'precision: ':precision,
    'recall: ' :recall,
    'f1_score: ':f1
          }
with open (os.path.join(LOG_PATH,LOG_METRICS_PKL),'wb') as output_file:
    pickle.dump(metrices2,output_file)

In [180]:
with mlflow.start_run(experiment_id=exp.experiment_id,
                      run_name=model2["model_description"]):
    # Log pickles
    mlflow.log_artifacts(LOG_PATH)

    # Track metrics
    mlflow.log_metric('precision', precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("accuracy", rmse1)

    mlflow.log_params({
        'k': 30,
        'min_k': 9,
        'sim_options': {'name': 'pearson', 'user_based': True}
    })

In [183]:
model1.predict('A34BZM6S9L7QI4', '1400501466', verbose=True)

user: A34BZM6S9L7QI4 item: 1400501466 r_ui = None   est = 4.25   {'actual_k': 0, 'was_impossible': False}


Prediction(uid='A34BZM6S9L7QI4', iid='1400501466', r_ui=None, est=4.246334308439121, details={'actual_k': 0, 'was_impossible': False})

In [187]:
# Predicting rating for a sample user with an interacted product
sample=test_data.sample(1).iloc[0]
model1.predict(sample["User_ID"], sample["Product_ID"], r_ui=sample["Ratings"], verbose=True)

user: APMYPD97EHUUZ item: B008D4X9UI r_ui = 4.00   est = 4.07   {'actual_k': 1, 'was_impossible': False}


Prediction(uid='APMYPD97EHUUZ', iid='B008D4X9UI', r_ui=4.0, est=4.0736609034101985, details={'actual_k': 1, 'was_impossible': False})

In [244]:
model1.predict('ATOKT8QYK967L','B005O7LJAE')

Prediction(uid='ATOKT8QYK967L', iid='B005O7LJAE', r_ui=None, est=4.161209623970819, details={'actual_k': 0, 'was_impossible': False})

In [189]:
user_item_interactions_matrix=df.pivot(index='User_ID',columns='Product_ID',values='Ratings')
user_item_interactions_matrix

Product_ID,1400501466,1400532655,9983891212,B00000DM9W,B00000J1V5,B00000JDF5,B00000JDF6,B00000K135,B00000K4KH,B00001P4XA,B00001P4XH,B00001P4ZH,B00001W0DI,B00001WRSJ,B00001ZWXA,B000021YU8,B0000228GG,B000026D8E,B00002EQCW,B00003006E,B00003006R,B000031KIM,B00003CWDG,B00003CWDH,B00003G1RG,B00004RC2D,B00004SABB,B00004SB92,B00004SY4H,B00004T8R2,B00004THCZ,B00004VX3T,B00004W3ZQ,B00004WCFT,B00004WCGF,B00004WCIC,B00004WCID,B00004XOM3,B00004Z0BN,B00004Z5D1,...,B00JNAA54O,B00JNBVK4M,B00JO6UGFU,B00JOI6FZ8,B00JOS04PK,B00JP12170,B00JP7R7DC,B00JQTNVL6,B00JR6GCZA,B00JTU88Y2,B00JWV1LP6,B00JWXTOIA,B00JX1ZS5O,B00JXFM75Y,B00JXUUVWU,B00JXVPHC8,B00JY4QCJQ,B00JZAB8OI,B00JZC972Q,B00JZM7TKI,B00K6ZAKCW,B00K91DB7Y,B00KFAGCUM,B00KH8PUDW,B00KHA2DQM,B00KHA5G6G,B00KIMX4EY,B00KJJW36G,B00KK9481I,B00KMRGB7C,B00KNM763E,B00KONCDVM,B00KVNY2KA,B00KXAFYZS,B00KYMCJF8,B00L21HC7A,B00L2442H0,B00L26YDA4,B00L3YHF6O,B00LGQ6HL8
User_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A100UD67AHFODS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A100WO06OQR8BQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A105S56ODHGJEK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A105TOJ6LTVMBG,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A10AFVU66A79Y1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZBXKUH4AIW3X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZCE11PSTCH1L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZMY6E8B52L2T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,NaN,NaN,5.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,NaN

In [209]:
# non_interacted_products=user_item_interactions_matrix.loc['A100UD67AHFODS'][user_item_interactions_matrix.loc['A100UD67AHFODS'].isnull()].index.tolist()
# non_interacted_products
# recommendations=[]
# for i in non_interacted_products:
#     # print(model1.predict('A100UD67AHFODS',i).est)
#     est=model1.predict('A100UD67AHFODS',i).est
#     recommendations.append((i,est))
# recommendations.sort(key=lambda x:x[1],reverse=True)    
# recommendations

[('B0052SCU8U', 4.973286611224055),
 ('B000N99BBC', 4.919623084419554),
 ('B001TH7T2U', 4.909452422148865),
 ('B008EQZ25K', 4.905809993334528),
 ('B003ES5ZUU', 4.901510778692227),
 ('B0036Q7MV0', 4.900330856675833),
 ('B00BQ4F9ZA', 4.897114642949376),
 ('B0000BZL1P', 4.894640769923062),
 ('B004ELA0SS', 4.870453988447825),
 ('B002V8C3W2', 4.8657505834987855),
 ('B001TH7GUU', 4.865150891842394),
 ('B000JV9LUK', 4.864968904457252),
 ('B00316263Y', 4.8617561140632715),
 ('B005ES0YYA', 4.861361619236413),
 ('B001UI2FPE', 4.853679587389569),
 ('B000M2TAN4', 4.852659203266192),
 ('B001QUA6RA', 4.8511086559331416),
 ('B0033PRWSW', 4.850227269112632),
 ('B001TH7GSW', 4.845303473237367),
 ('B0002LEMWE', 4.8443746047367355),
 ('B00IVPU6AA', 4.844217743840106),
 ('B000NP3DJW', 4.843558141600538),
 ('B00D6XW62I', 4.843342205926016),
 ('B002JQNXZC', 4.8395606287358),
 ('B003ES5ZR8', 4.838038620735075),
 ('B001342KM8', 4.8369896684581875),
 ('B0019EHU8G', 4.836119774622616),
 ('B003L1ZYZ6', 4.8354384

In [238]:
def get_recommendations(data,user_id,top_n,model):
    user_item_interactions_matrix=df.pivot(index='User_ID',columns='Product_ID',values='Ratings')

    non_interacted_products=user_item_interactions_matrix.loc[user_id][user_item_interactions_matrix.loc[user_id].isnull()].index.tolist()
    recommendations=[]
    for i in non_interacted_products:
        est=model.predict('user_id',i).est
        recommendations.append((i,est))
    recommendations.sort(key=lambda x:x[1],reverse=True)    
    return recommendations[:top_n+1]

In [242]:
get_recommendations(df, 'A3LDPF5FMB782Z', 20, model1)

[('B0052SCU8U', 4.836800187278955),
 ('B000N99BBC', 4.783136660474454),
 ('B001TH7T2U', 4.772965998203765),
 ('B008EQZ25K', 4.769323569389428),
 ('B003ES5ZUU', 4.765024354747127),
 ('B0036Q7MV0', 4.763844432730733),
 ('B00BQ4F9ZA', 4.760628219004276),
 ('B0000BZL1P', 4.758154345977962),
 ('B004ELA0SS', 4.733967564502725),
 ('B002V8C3W2', 4.7292641595536855),
 ('B001TH7GUU', 4.7286644678972936),
 ('B000JV9LUK', 4.728482480512152),
 ('B00316263Y', 4.725269690118171),
 ('B005ES0YYA', 4.724875195291313),
 ('B001UI2FPE', 4.717193163444469),
 ('B000M2TAN4', 4.716172779321092),
 ('B001QUA6RA', 4.7146222319880415),
 ('B0033PRWSW', 4.713740845167532),
 ('B001TH7GSW', 4.708817049292267),
 ('B0002LEMWE', 4.7078881807916355),
 ('B00IVPU6AA', 4.707731319895006)]